In [1]:
import numpy
import torch
import gym 
import imageio
from stable_baselines3 import PPO
from stable_baselines3 import PPO 

In [ ]:

show = False

if not show :env = gym.make("CartPole-v1", render_mode="rgb_array")
else: env = gym.make("CartPole-v1",render_mode="human")



model = PPO("MlpPolicy", env, verbose=1, device='cuda')
model.learn(total_timesteps=10000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Python312\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
c:\Python312\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 24.1     |
|    ep_rew_mean     | 24.1     |
| time/              |          |
|    fps             | 2936     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 28.5        |
|    ep_rew_mean          | 28.5        |
| time/                   |             |
|    fps                  | 1617        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008630529 |
|    clip_fraction        | 0.0966      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.0015     |
|    learning_rate        | 0.

In [4]:
custom_rewards = []
episodes = 10
frames = []

for episode in range(episodes):
    state = env.reset()[0]
    done = False
    episode_reward = 0

    while not done:

        if not show and episode == 0: frames.append(env.render())
        action, _ = model.predict(state, deterministic=True)
        state, reward, done, truncated, _ = env.step(action)
        episode_reward += reward

    custom_rewards.append(episode_reward)

avg_custom_reward = sum(custom_rewards) / episodes
print(f"Average reward : {avg_custom_reward}")
# Save frames as a GIF
imageio.mimsave('cartpole_agent_part1.gif', frames, fps=30)
print("GIF saved as 'cartpole_agent_part1.gif'")

Average reward : 525.2
GIF saved as 'cartpole_agent_part1.gif'


In [8]:
import numpy as np
from gym import RewardWrapper

class CustomCartPoleReward(RewardWrapper):
    def __init__(self, env):
        super(CustomCartPoleReward, self).__init__(env)

    def reward(self, reward):
        # Increase reward for keeping the pole upright and penalize for moving away from the center
        x, x_dot, theta, theta_dot = self.env.state
        new_reward = reward - np.abs(theta)  # Penalize for angle from upright
        return new_reward

# Initialize the custom environment
if not show: custom_env = CustomCartPoleReward(gym.make("CartPole-v1", render_mode="rgb_array"))
else: custom_env = CustomCartPoleReward(gym.make("CartPole-v1", render_mode="human"))

In [9]:

custom_model = PPO("MlpPolicy", custom_env, verbose=1, device='cuda')
custom_model.learn(total_timesteps=10000)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 23.3     |
|    ep_rew_mean     | 21.4     |
| time/              |          |
|    fps             | 3233     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 25.8        |
|    ep_rew_mean          | 23.6        |
| time/                   |             |
|    fps                  | 1671        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009037672 |
|    clip_fraction        | 0.122       |
|    clip_range           | 0.2         |
|    entropy_loss   

In [12]:
custom_rewards = []
episodes = 10
frames = []

for episode in range(episodes):
    state = custom_env.reset()[0]
    done = False
    episode_reward = 0

    while not done:
        if not show and episode == 0: frames.append(custom_env.render())
        action, _ = custom_model.predict(state, deterministic=True)
        state, reward, done, truncated, _ = custom_env.step(action)
        episode_reward += reward

    custom_rewards.append(episode_reward)

avg_custom_reward = sum(custom_rewards) / episodes
print(f"Average reward with custom reward function: {avg_custom_reward}")

imageio.mimsave('cartpole_agent_part2.gif', frames, fps=30)
print("GIF saved as 'cartpole_agent_part2.gif'")

Average reward with custom reward function: 464.4636284861434
GIF saved as 'cartpole_agent_part2.gif'


In [26]:
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch as th
from torch import nn
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed

class CustomMLP(nn.Module):
    def __init__(self):
        super(CustomMLP, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(4, 64), nn.ReLU(),
            nn.Linear(64, 64), nn.ReLU(),
            nn.Linear(64, 64), nn.ReLU(),
            nn.Linear(64, 64), nn.ReLU(),
            nn.Linear(64, 2)
        ).to(th.device('cuda' if th.cuda.is_available() else 'cpu'))

    def forward(self, x):
        return self.network(x)

# Modify the policy architecture
model = PPO("MlpPolicy", env, policy_kwargs=dict(activation_fn=th.nn.ReLU, net_arch=[128, 128]), verbose=1, device='cuda')

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [27]:
model.learn(total_timesteps=10000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 23.5     |
|    ep_rew_mean     | 23.5     |
| time/              |          |
|    fps             | 3144     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 26.8        |
|    ep_rew_mean          | 26.8        |
| time/                   |             |
|    fps                  | 1710        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010400737 |
|    clip_fraction        | 0.0844      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | 0.00899     |
|    learning_rate        | 0.

In [29]:
custom_rewards = []
episodes = 10
frames = []
for episode in range(episodes):
    state = env.reset()[0]
    done = False
    episode_reward = 0

    while not done:
        if not show and episode == 0: frames.append(env.render())
        action, _ = model.predict(state, deterministic=True)
        state, reward, done, truncated, _ = env.step(action)
        episode_reward += reward

    custom_rewards.append(episode_reward)

avg_custom_reward = sum(custom_rewards) / episodes
print(f"Average reward : {avg_custom_reward}")


imageio.mimsave('cartpole_agent_part3.gif', frames, fps=30)
print("GIF saved as 'cartpole_agent_part3.gif'")

Average reward : 402.4
GIF saved as 'cartpole_agent_part3.gif'
